# Intro
Name:  

    ORACLES_Build_DARE

Purpose:  

    Build the aerosol radiative effect input files from the SSFR reflectances, 4STAR AOD, and 4STAR skyscan results
  
Input:

    none at command line
  
Output:

    figures and save files...
  
Keywords:

    none
  
Dependencies:

    - Sp_parameters.py : for Sp class definition, and for defining the functions used to build parameters
    - matplotlib
    - numpy
    - scipy : for saving and reading
    - plotting_utils (user defined plotting routines)
    - hdf5storage
  
Needed Files:

  - file.rc : for consistent creation of look of matplotlib figures
  - 4STAR_cloud retrieval .mat files
  
 Modification History:
 
     Written: by Samuel LeBlanc, Santa Cruz, CA, 2019-12-02

# Import of modules

In [128]:
import numpy as np
import hdf5storage as hs
import os
import write_utils as wu
import scipy.io as sio
from path_utils import getpath
import matplotlib.pyplot as plt
import load_utils as lu
from write_utils import nearest_neighbor, iterate_dict_unicode

In [2]:
%matplotlib notebook

In [86]:
from tqdm.notebook import tqdm 
from datetime import datetime

In [4]:
import Run_libradtran as Rl

In [5]:
name = 'ORACLES'

In [6]:
vv = 'v1'
vr = 'R3'

In [15]:
fp = getpath(name)
fp_rtm = getpath('rtm')
fp_uvspec = getpath('uvspec_bin')+'uvspec'
matfile = fp+'{}_all_cld_ict.mat'.format(vr)
fp_uvspec_dat = getpath('uvspec_dat')
fp_rtmdat = fp_rtm+'dat/'

Return path named:  ORACLES /data/sam/ORACLES/
Return path named:  rtm /home/sam/rtm/
Return path named:  uvspec_bin /home/sam/libradtran/libRadtran-2.0.2/bin/
Return path named:  uvspec_dat /home/sam/libradtran/libRadtran-2.0.2/data/


# Load the files

## Load the 4STAR AOD

In [16]:
ar = hs.loadmat(fp+'/aod_ict/v8/{v}/all_aod_ict_{v}_2016.mat'.format(v=vr))

In [17]:
ar.keys()

[u'AOD1040',
 u'AOD0532',
 u'AOD0530',
 u'UNCAOD0550',
 u'UNCAOD0700',
 u'AOD0355',
 u'AOD1627',
 u'AOD0520',
 u'UNCAOD0620',
 u'AOD0606',
 u'AOD1064',
 u'Latitude',
 u'UNCAOD0452',
 u'fl_alt_6',
 u'AOD0620',
 u'AOD0700',
 u'AOD0550',
 u'AOD0660',
 u'UNCAOD1627',
 u'AOD0452',
 u'fl_routine',
 u'AOD0470',
 u'fl1',
 u'UNCAOD1236',
 u'fl3',
 u'fl2',
 u'UNCAOD1064',
 u'UNCAOD0501',
 u'AOD_angstrom_470_865',
 u'AOD_polycoef_a0',
 u'AOD_polycoef_a2',
 u'UNCAOD0606',
 u'UNCAOD0355',
 u'qual_flag',
 u'UNCAOD0675',
 u'AOD1559',
 u'UNCAOD1650',
 u'UNCAOD1559',
 u'UNCAOD1250',
 u'AOD0501',
 u'Longitude',
 u'AOD_polycoef_a1',
 u'UNCAOD0660',
 u'GPS_Alt',
 u'AOD1250',
 u'flac',
 u'UNCAOD0380',
 u'fl',
 u'AOD0675',
 u'fl_QA',
 u'AOD1236',
 u'AOD1650',
 u'UNCAOD0470',
 u'AOD0380',
 u'UNCAOD0865',
 u'Start_UTC',
 u'flr',
 u'UNCAOD0781',
 u'days',
 u'UNCAOD0520',
 u'fl_alt_22',
 u'AOD0781',
 u'fl_alt_18',
 u'flag_acaod',
 u'fl_alt',
 u'AOD0865',
 u'amass_aer',
 u'flacr',
 u'UNCAOD1020',
 u'UNCAOD0532',

In [18]:
ar['AOD0501'].shape

(483692,)

In [19]:
sza = np.arccos(1.0/ar['amass_aer'])*180.0/np.pi

/home/sam/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in arccos
  """Entry point for launching an IPython kernel.


In [20]:
days = ['20160824','20160825','20160827','20160830','20160831','20160902','20160904','20160906','20160908',
       '20160910','20160912','20160914','20160918','20160920','20160924','20160925','20160927','20160930']

## Load the retrieved Cloud properties

In [21]:
cl = hs.loadmat(fp+'data_other/ssfr_2016_retrieved_COD.mat')

In [22]:
cl.keys()

[u'tau',
 u'aod',
 u'utc',
 u'sza',
 u'lon',
 u'days',
 u'a1',
 u'a0',
 u'a2',
 u'lat',
 u'ref']

In [23]:
cl['tau'].shape

(380923,)

In [24]:
dds = ['20160830','20160831','20160902','20160904','20160906','20160908',
       '20160910','20160912','20160914','20160918','20160920','20160924','20160925','20160927']

In [25]:
cl['days']

array([ 0.,  0.,  0., ..., 13., 13., 13.])

In [26]:
dd = np.unique(cl['days'])

In [27]:
cod,ref = [],[]
for d in dd:
    print d
    fld = cl['days']==d
    fad = ar['days']==d+3.0
    #nearest neighbor, but not more than a minute away
    cod_tmp = nearest_neighbor(cl['utc'][fld],cl['tau'][fld],ar['Start_UTC'][fad],dist=1.0/60.0) 
    ref_tmp = nearest_neighbor(cl['utc'][fld],cl['ref'][fld],ar['Start_UTC'][fad],dist=1.0/60.0)
    cod = np.append(cod,cod_tmp)
    ref = np.append(ref,ref_tmp)

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0


In [28]:
cod.shape

(379755,)

In [29]:
len(dds)

14

In [30]:
len(np.unique(ar['days']))

18

## Load the skyscan retrievals

In [52]:
try:
    ae,ae_dict = lu.load_netcdf(fp+'aeroinv_2016/netcdf4/4STAR-aeroinv_P3_2016_R0.nc',everything=True)
except:
    import h5py as h5
    f5 = h5.File(fp+'aeroinv_2016/netcdf4/4STAR-aeroinv_P3_2016_R0.nc','r')
    ae5 = {}
    for ka,kd in f5.iteritems():
        ae5[ka] = kd.value
    ae = ae5

Reading file: /data/sam/ORACLES/aeroinv_2016/netcdf4/4STAR-aeroinv_P3_2016_R0.nc


IOError: [Errno -101] NetCDF: HDF error: '/data/sam/ORACLES/aeroinv_2016/netcdf4/4STAR-aeroinv_P3_2016_R0.nc'

In [80]:
ke = ae.keys()
ke.sort()
ke

[u'AAOD',
 u'AGOD',
 u'AOD_fit_coarse',
 u'AOD_fit_fine',
 u'AOD_fit_total',
 u'AOD_meas',
 u'GPS_Altitude',
 u'GPS_Altitude_std',
 u'Heading',
 u'Heading_std',
 u'Lat',
 u'Lon',
 u'PF_angle',
 u'PF_coarse',
 u'PF_fine',
 u'PF_total',
 u'PWV',
 u'P_static',
 u'Pitch',
 u'Pitch_std',
 u'QA_level',
 u'RH',
 u'Roll',
 u'Roll_std',
 u'SA',
 u'SAZ',
 u'SEL',
 u'SSA',
 u'SZA',
 u'TOD_fit',
 u'TOD_meas',
 u'TOD_meas_minus_fit',
 u'T_static',
 u'base_time',
 u'g_coarse',
 u'g_fine',
 u'g_total',
 u'm_ray',
 u'n_imag',
 u'n_real',
 u'normalized_sky_radiance',
 u'normalized_sky_radiance_fit',
 u'psd',
 u'radius',
 u'sca_angle',
 u'scan_tag',
 u'scan_type',
 u'sfc_alb',
 u'sky_radiance_fit_error',
 u'sphericity',
 u'sphericity_err',
 u'time',
 u'time_offset',
 u'wavelength']

In [81]:
ae['AOD_meas'][0]

array([0.2365, 0.1788, 0.1115, 0.0665, 0.0549])

In [33]:
ae_dict['SSA']

<type 'netCDF4._netCDF4.Variable'>
float64 SSA(time, wavelength)
    long_name: single scattering albedo
    units: unitless
    source: retrieval
    values: 1.0 1.5 2.0
    lev2: Has 21+ values at all wavelengths with scattering angle >80 deg and multiple points in each of 4 scattering angle ranges, and <20% asymmetry between CW and CCW legs
    lev1p5: Has 10+ values at all wavelengths with scattering angle >80 deg and at least one point in each of 4 scattering angle ranges
    lev1: Has at least one point at all wavelengths in each of 4 scattering angle ranges
    Comment: Based on AERONET inversion criteria in https://aeronet.gsfc.nasa.gov/new_web/Documents/AERONETcriteria_final1.pdf
    Comment2: A consequence of the symmetry requirement for level 2 is that PPL scans can assess at no higher than 1.5
unlimited dimensions: time
current shape = (84, 5)
filling on, default _FillValue of 9.96920996839e+36 used

In [82]:
ae['SSA'].shape

(84, 5)

In [194]:
ae_dict['time']

<type 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: Time offset from midnight
    units: seconds since 2016-08-31 00:00:00 0:00
unlimited dimensions: time
current shape = (84,)
filling on, default _FillValue of 9.96920996839e+36 used

In [83]:
ae['time']/3600.0

array([  9.36436349,   9.38314424,  11.44100489,  13.53473135,
        13.55286422,  13.63317105,  13.65609344,  14.15397498,
        14.17291726,  55.12454682,  56.45493072,  56.47428852,
        56.50556153,  56.52408765,  56.62796449,  56.64651974,
        57.3145811 ,  57.57097029,  57.58955459,  58.57214188,
        58.99556736,  60.49512862,  60.51375567,  60.53747322,
        62.83647775, 153.56740963, 153.58586041, 154.09168935,
       154.11113343, 154.98052584, 155.82233424, 155.84067681,
       156.42516806, 156.44378445, 157.14872684, 157.16740038,
       203.2280269 , 251.08488591, 252.31515166, 300.24803314,
       301.14747711, 301.1660435 , 301.18294843, 301.7608256 ,
       301.77941441, 302.12064148, 302.14029531, 345.31344008,
       345.33189656, 346.52380833, 346.63178176, 346.6632354 ,
       349.91946379, 441.84949381, 489.40488159, 489.42472078,
       489.45796597, 489.48503416, 489.52466494, 489.55642007,
       489.58136179, 491.02055947, 491.14471216, 492.66

In [84]:
days = ['20160824','20160825','20160827','20160830','20160831','20160902','20160904','20160906','20160908',
       '20160910','20160912','20160914','20160918','20160920','20160924','20160925','20160927','20160930']

In [87]:
ar['doy'] = np.array([datetime.strptime(days[int(d)],'%Y%m%d').timetuple().tm_yday for d in ar['days']])

In [88]:
datetime.strptime(days[4],'%Y%m%d').timetuple().tm_yday

244

In [89]:
ar['time_ae'] = ar['Start_UTC']+(24.0*(ar['doy']-244))

In [90]:
ar['time_ae']

array([-156.14305556, -156.14277778, -156.1425    , ...,  737.93      ,
        737.93027778,  737.93055556])

# Prepare the base dict and defaults

In [91]:
from datetime import datetime
datetime(2015,11,17).timetuple().tm_yday

321

In [92]:
# for all 4STAR aerosol arrays
fla = (ar['flag_acaod']==1) & ar['fl'] & ar['fl_QA'] & (ar['days']>2.0)

In [93]:
# for the cod and ref arrays
fld = (ar['days']>2.0) & (ar['days']!=17.0) 
flb = (ar['flag_acaod'][fld]==1) & ar['fl'][fld] & ar['fl_QA'][fld]

In [94]:
ka = ar.keys()
ka.sort()
ka

[u'AOD0355',
 u'AOD0380',
 u'AOD0452',
 u'AOD0470',
 u'AOD0501',
 u'AOD0520',
 u'AOD0530',
 u'AOD0532',
 u'AOD0550',
 u'AOD0606',
 u'AOD0620',
 u'AOD0660',
 u'AOD0675',
 u'AOD0700',
 u'AOD0781',
 u'AOD0865',
 u'AOD1020',
 u'AOD1040',
 u'AOD1064',
 u'AOD1236',
 u'AOD1250',
 u'AOD1559',
 u'AOD1627',
 u'AOD1650',
 u'AOD_angstrom_470_865',
 u'AOD_polycoef_a0',
 u'AOD_polycoef_a1',
 u'AOD_polycoef_a2',
 u'GPS_Alt',
 u'Latitude',
 u'Longitude',
 u'Start_UTC',
 u'UNCAOD0355',
 u'UNCAOD0380',
 u'UNCAOD0452',
 u'UNCAOD0470',
 u'UNCAOD0501',
 u'UNCAOD0520',
 u'UNCAOD0530',
 u'UNCAOD0532',
 u'UNCAOD0550',
 u'UNCAOD0606',
 u'UNCAOD0620',
 u'UNCAOD0660',
 u'UNCAOD0675',
 u'UNCAOD0700',
 u'UNCAOD0781',
 u'UNCAOD0865',
 u'UNCAOD1020',
 u'UNCAOD1040',
 u'UNCAOD1064',
 u'UNCAOD1236',
 u'UNCAOD1250',
 u'UNCAOD1559',
 u'UNCAOD1627',
 u'UNCAOD1650',
 u'amass_aer',
 u'days',
 'doy',
 u'fl',
 u'fl1',
 u'fl2',
 u'fl3',
 u'fl_QA',
 u'fl_alt',
 u'fl_alt_18',
 u'fl_alt_22',
 u'fl_alt_6',
 u'fl_routine',
 u'flac

In [95]:
doy = datetime.strptime(dds[int(ar['days'][fla][0])],'%Y%m%d').timetuple().tm_yday

In [96]:
doy

250

In [112]:
geo = {'lat':ar['Latitude'][0],'lon':ar['Longitude'][0],'doy':doy,'zout':[0,1.5,100.0]}
aero_no = {} # none
cloud = {'ztop':1.0,'zbot':0.5,'write_moments_file':False}
source = {'wvl_range':[201.0,4900.0],'source':'solar','integrate_values':True,'run_fuliou':True,
          'dat_path':fp_uvspec_dat}
albedo = {'create_albedo_file':False,'sea_surface_albedo':True,'wind_speed':5.0}

In [113]:
cloud['phase'] = 'wc'
geo['sza'] = 40.0
cloud['tau'] = 2.0
cloud['ref'] = 5.0
pmom = Rl.make_pmom_inputs(fp_rtm=fp_rtmdat,source='solar',deltascale=False)
cloud['moms_dict'] = pmom

In [114]:
pmom['wvl'][0] = 0.250

In [115]:
wvl = np.append(np.append([250.0],ae['wavelength']),4900.0)
wvl

array([ 250.,  400.,  500.,  675.,  870.,  995., 4900.])

In [116]:
aero = {'expand_hg':True,'disort_phase':False,'z_arr':[2.0,5.0],
        'wvl_arr':wvl}

In [117]:
def fx_aero(aprop):
    'Function the aerosol property a 2d matrix for height and spectra, and extend the wavelength from 250 to 4900 nm'
    atmp = np.append([aprop[0]],np.append(aprop,aprop[-1]))
    return np.array([atmp,atmp])

In [118]:
def fx_ext(a0,a1,a2,wvl=wvl):
    'Function to create the extinction coefficients from 4STAR AODs'
    aod = np.exp(np.polyval([a2,a1,a0],np.log(wvl)))
    aod[-1] = 0.0 # set the last wavelength to zero
    return np.array([aod/3.0,aod*0.0])

In [119]:
aero['ext'] = fx_ext(ar['AOD_polycoef_a0'][fla][0],ar['AOD_polycoef_a1'][fla][0],ar['AOD_polycoef_a2'][fla][0])

In [120]:
aero['asy'] = fx_aero(ae['g_total'][0])

In [121]:
aero['ssa'] = fx_aero(ae['SSA'][0])

## Prepare the file list and saving

In [122]:
def isjupyter():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

In [123]:
# open the list file
f = open(fp_rtm+'{}_DARE_{}.sh'.format(name,vv),'w')
fpp_in = fp_rtm+'input/{}_DARE_{}/'.format(name,vv)
fpp_out = fp_rtm+'output/{}_DARE_{}/'.format(name,vv)

In [124]:
if not os.path.isdir(fpp_in):
    os.mkdir(fpp_in)
if not os.path.isdir(fpp_out):
     os.mkdir(fpp_out)

## Write the files

In [120]:
# for writing out the files

In [125]:
if isjupyter():
    pbar = tqdm(total=len(ar['Start_UTC'][fla]))
for i,u in enumerate(ar['Start_UTC'][fla]):
    
    f_in = '{name}_{vv}_DARE_{i:03d}_withaero.dat'.format(name=name,vv=vv,i=i)

    geo['lat'],geo['lon'],geo['sza'] = ar['Latitude'][fla][i],ar['Longitude'][fla][i],sza[fla][i]
    day = days[ar['days'][fla][i].astype(int)]
    geo['doy'] = datetime(int(day[0:4]),int(day[4:6]),int(day[6:])).timetuple().tm_yday

    cloud['tau'],cloud['ref'] = cod[flb][i],ref[flb][i]
    cloud['write_moments_file'] = True

    iae = np.argmin(abs(ar['time_ae'][fla][i]-ae['time']/3600.0))

    # Only run for aerosol rertievals within 1 hour
    if abs(ar['time_ae'][fla][i]-ae['time']/3600.0)[iae]<1.0: 

        aero['ext'] = fx_ext(ar['AOD_polycoef_a0'][fla][i],ar['AOD_polycoef_a1'][fla][i],ar['AOD_polycoef_a2'][fla][i])
        aero['ssa'] = fx_aero(ae['SSA'][0])
        aero['asy'] = fx_aero(ae['g_total'][iae])

        Rl.write_input_aac(fpp_in+f_in,geo=geo,aero=aero,cloud=cloud,source=source,albedo=albedo,
                                   verbose=False,make_base=False,set_quiet=True)
        f.write('{uv} < {fin} > {out}\n'.format(uv=fp_uvspec,fin=fpp_in+f_in,out=fpp_out+f_in))

        f_in = '{name}_{vv}_star_{i:03d}_noaero.dat'.format(name=name,vv=vv,i=i)
        Rl.write_input_aac(fpp_in+f_in,geo=geo,aero=aero_no,cloud=cloud,source=source,albedo=albedo,
                                   verbose=False,make_base=False,set_quiet=True)
        f.write('{uv} < {fin} > {out}\n'.format(uv=fp_uvspec,fin=fpp_in+f_in,out=fpp_out+f_in))

    if isjupyter(): 
        pbar.update(1)
    else:
        print i

f.close()

wvl_range starting too low, setting to 250 nm


Run the files from command line:

using command parallel --jobs=20 < ORACLES_DARE_v1.sh

In [129]:
f_list = fp_rtm+'{}_DARE_{}.sh'.format(name,vv)

In [130]:
! wc -l $f_list

41134 /home/sam/rtm/ORACLES_DARE_v1.sh


In [ ]:
!parallel --job=20 --bar < $f_list

## Read the files

In [133]:
n = len(ar['Start_UTC'][fla])
nz = len(geo['zout'])
nw = len(aero['wvl_arr'])

if isjupyter():
    pbar = tqdm(total=n)
    
dat = {'cod':np.zeros(n)+np.nan,'ref':np.zeros(n)+np.nan,'ext':np.zeros((n,nw))+np.nan,
       'ssa':np.zeros((n,nw))+np.nan,'asy':np.zeros((n,nw))+np.nan,'zout':geo['zout'],
       'wvl':aero['wvl_arr'],
       'dn':np.zeros((n,nz))+np.nan,'up':np.zeros((n,nz))+np.nan,
       'dn_noa':np.zeros((n,nz))+np.nan,'up_noa':np.zeros((n,nz))+np.nan}
for i,u in enumerate(ar['Start_UTC'][fla]):
    
    dat['cod'][i] = cod[flb][i]
    dat['ref'][i] = ref[flb][i]

    iae = np.argmin(abs(ar['time_ae'][fla][i]-ae['time']/3600.0))
    # Only run for aerosol rertievals within 1 hour
    if abs(ar['time_ae'][fla][i]-ae['time']/3600.0)[iae]<1.0: 

        dat['ext'][i,:] = fx_ext(ar['AOD_polycoef_a0'][fla][i],ar['AOD_polycoef_a1'][fla][i],ar['AOD_polycoef_a2'][fla][i])[0]
        dat['ssa'][i,:] = fx_aero(ae['SSA'][0])[0]
        dat['asy'][i,:] = fx_aero(ae['g_total'][iae])[0]
        try:
            f_in = '{name}_{vv}_DARE_{i:03d}_withaero.dat'.format(name=name,vv=vv,i=i)
            o = Rl.read_libradtran(fpp_out+f_in,zout=geo['zout'])
            f_in = '{name}_{vv}_star_{i:03d}_noaero.dat'.format(name=name,vv=vv,i=i)
            on = Rl.read_libradtran(fpp_out+f_in,zout=geo['zout'])

            dat['dn'][i,:] = o['diffuse_down']+o['direct_down']
            dat['dn_noa'][i,:] = on['diffuse_down']+on['direct_down']
            dat['up'][i,:] = o['diffuse_up']
            dat['up_noa'][i,:] = on['diffuse_up']
        except:
            pass

    if isjupyter(): 
        pbar.update(1)
    else:
        print i


In [134]:
dat['dare'] = (dat['dn']-dat['up']) - (dat['dn_noa']-dat['up_noa'])

## Save the file

In [135]:
dat1 = iterate_dict_unicode(dat)
print 'saving file to: '+fp+'{name}_DARE_{vv}.mat'.format(name=name,vv=vv)
hs.savemat(fp+'{name}_DARE_{vv}.mat'.format(name=name,vv=vv),dat1)

no dn
no asy
no up
no up_noa
no wvl
no ssa
no dare
no dn_noa
zout
no ext
no cod
no ref
saving file to: /data/sam/ORACLES/ORACLES_DARE_v1.mat


/home/sam/.local/lib/python2.7/site-packages/hdf5storage/__init__.py:1306: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(filename)
